In [ ]:
#query for efi2 deficit for motor neuron disease, using updated code for efi2 deficits
 CREATE or replace TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efi2_mnd` as (
#bring all snoemd and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    cast(j.first_episodestartdate as datetime) first_episodestartdate
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#filter for mnd snomed codes
  b as (
    select
    person_id,
    snomedcode,
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Motor neuron disease') AND dateevent < first_episodestartdate),
#filter for mnd ctv3 codes
  c as (
    select
    person_id,
    ctv3code, 
    dateevent
    from a_master
    where ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Motor neuron disease') AND dateevent < first_episodestartdate),
#union distinct individuals identified from ctv3 and snomed ct codes together
  snomed_ctv3 as (
    select
    * from b

    union distinct

    select *
    from c)
#apply dummy variable
    select
    distinct person_id,
    case when dateevent IS NOT NULL then 'yes'
    end as mnd
    from snomed_ctv3)
  



